# Evaluate model endpoints using Prompt Flow Eval APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [promptflow-evals](https://microsoft.github.io/promptflow/reference/python-library-reference/promptflow-evals/promptflow.html)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using promptflow-evals

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install promptflow-evals
%pip install promptflow-azure

### Parameters and imports

In [ ]:
from pprint import pprint

import pandas as pd
import random

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [ ]:
env_var = {
    "gpt4-0613": {
        "endpoint": "https://ai-***.**.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "gpt35-turbo": {
        "endpoint": "https://ai-**.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "mistral7b": {
        "endpoint": "https://***.eastus.inference.ml.azure.com/v1/chat/completions",
        "key": "***",
    },
    "phi3_mini_serverless": {
        "endpoint": "https://Phi-3-mini-4k-instruct-rpzhe.eastus2.models.ai.azure.com/v1/chat/completions",
        "key": "***",
    },
    "tiny_llama": {
        "endpoint": "https://api-inference.huggingface.co/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/v1/chat/completions",
        "key": "***",
    },
    "gpt2": {
        "endpoint": "https://api-inference.huggingface.co/models/openai-community/gpt2",
        "key": "***",
    },
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [ ]:
azure_ai_project = {"subscription_id": "***", "resource_group_name": "***", "project_name": "***"}

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [ ]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [ ]:
from promptflow.core import AzureOpenAIModelConfiguration

configuration = AzureOpenAIModelConfiguration(
    azure_endpoint="https://ai-***.openai.azure.com",
    api_key="**",
    api_version="2023-03-15-preview",
    azure_deployment="gpt-35-turbo-16k",
)

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [ ]:
from app_target import ModelEndpoints
import pathlib

from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)


content_safety_evaluator = ContentSafetyEvaluator(project_scope=azure_ai_project)
relevance_evaluator = RelevanceEvaluator(model_config=configuration)
coherence_evaluator = CoherenceEvaluator(model_config=configuration)
groundedness_evaluator = GroundednessEvaluator(model_config=configuration)
fluency_evaluator = FluencyEvaluator(model_config=configuration)
similarity_evaluator = SimilarityEvaluator(model_config=configuration)

models = [
    "gpt4-0613",
    "gpt35-turbo",
    "mistral7b",
    "phi3_mini_serverless",
    "tiny_llama",
    "gpt2",
]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project,
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "content_safety": content_safety_evaluator,
            "coherence": coherence_evaluator,
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,
        },
        evaluator_config={
            "content_safety": {"question": "${data.question}", "answer": "${target.answer}"},
            "coherence": {"answer": "${target.answer}", "question": "${data.question}"},
            "relevance": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "groundedness": {
                "answer": "${target.answer}",
                "context": "${data.context}",
                "question": "${data.question}",
            },
            "fluency": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
            "similarity": {"answer": "${target.answer}", "context": "${data.context}", "question": "${data.question}"},
        },
    )

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])